In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.losses import *
from keras.preprocessing.image import ImageDataGenerator
import skimage.io as io
import skimage.transform as tr
import skimage.color
from sklearn.metrics import classification_report
from glob import glob

## Own Scripts
from helper import * 
from loss_metrics import *
from unet import *


In [4]:
# initialise train data
train_size = 18300
val_size = 3922
test_size = 3922
batch_size = 32
dsPath = '/home/dan/Desktop/Datenset_Tower (Augmented)'
epochs = 40

In [5]:
# create generators for training 

trainInputPath = dsPath + '/train/images'
trainOutputPath = dsPath + '/train/masks'

valInputPath = dsPath + '/validate/images'
valOutputPath = dsPath + '/validate/masks'

trainGen = loadData(trainInputPath, trainOutputPath, batch_size)
valGen   = loadData(valInputPath, valOutputPath, batch_size)

In [6]:
# initialise new model
model = None
model = UNet((224,224,1), 1, 16, 5, 2.0)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 16) 160         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 224, 224, 16) 2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 112, 112, 16) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [8]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [7]:
# load trained Model  
from keras.models import load_model 
model = load_model('ds_step08.h5', custom_objects={'iou_loss': iou_loss})
model.summary()

OSError: Unable to open file (unable to open file: name = 'ds_step08.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# create callbacks
earlyStop = EarlyStopping(monitor='val_loss', patience = 5)
checkpoint = ModelCheckpoint(dsPath + '/results/training02.h5', save_best_only=True)
tensorBoard = keras.callbacks.TensorBoard(log_dir= dsPath + '/logs', histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

In [ ]:
# compile the model
model.compile(optimizer=Adam(lr=0.0001), loss=iou_loss, metrics=[f1, iou, precision, recall, error])

In [ ]:
# train and validate the model 
result = model.fit_generator(trainGen, steps_per_epoch= train_size/batch_size, epochs=epochs, validation_data=valGen, validation_steps=val_size/batch_size, verbose=1, shuffle=True, callbacks=[earlyStop, checkpoint])

In [ ]:
model.save(dsPath + '/results/training_01.h5')

In [ ]:
# list all data in history

print(result.history.keys())
# summarize history for accuracy
plt.plot(result.history['acc'])
plt.plot(result.history['val_acc'])
plt.title('model accuracy') #Benennung des Grafen
plt.ylabel('accuracy') #Benennung der y-Achse
plt.xlabel('epoch')    #Benennung der x-Achse
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('model loss') #Benennung des Grafen
plt.ylabel('loss')  #Benennung der y-Achse
plt.xlabel('epoch') #Benennung der x-Achse
plt.legend(['train', 'test'], loc='upper left')
plt.show()

print(result.history)

In [ ]:
# evaluate on the test dataset
testInputPath = dsPath + '/test/images'
testOutputPath = dsPath + '/test/masks'

testGen = loadData(testInputPath, testOutputPath, batch_size)

In [ ]:
model.evaluate_generator(testGen, steps=test_size/batch_size ,verbose=1 )

In [ ]:
predict = model.predict_generator(testGen, steps=test_size/batch_size, verbose=1)

In [ ]:
print (predict.shape)

In [ ]:
b = 0
i = 9
plt.imshow(predict[i, ..., 0], cmap = 'gray')
plt.show()
#plt.imshow(testGen[b][i, ..., 0], cmap='gray')
#plt.show()